In [1]:
!pip install opentree

In [1]:
from opentree import OT
import copy

## Bats
https://tree.opentreeoflife.org/opentree/argus/ottol@574724/Chiroptera

In [2]:
output = OT.synth_subtree('ott574724', label_format="name_and_id")
output.tree.print_plot(width=100)
output.tree.write(path="bats.tre",schema="newick")



                                 /-- Kerivoula hardwickii ott6790                                   
                                /+                                                                  
                                ||/- Kerivoula titania ott1010682                                   
                                |\+                                                                 
                               /+ \- Kerivoula kachinensis ott1022371                               
                               ||                                                                   
                               || /- Kerivoula intermedia ott183380                                 
                              /+\-+                                                                 
                              ||  \- Kerivoula minuta ott1025538                                    
                              ||                                                           

In [3]:
# Don't forget to cite your friendly phylogeneticists!
studies = output.response_dict['supporting_studies']
cites = OT.get_citations(studies) #this can be a bit slow
fi = open("bat_cites.txt","w")
fi.write(cites)
fi.close()

In [6]:
len(studies)

9

In [4]:
print(cites)

https://tree.opentreeoflife.org/curator/study/view/pg_1634?tab=trees&tree=tree3303
Agnarsson, I., Zambrana-torrelio C.M., Flores saldaña N.P., & May-collado L.M. 2011. A time-calibrated species-level phylogeny of bats (Chiroptera, Mammalia). PLoS Currents: Tree of Life, Version 44.
http://dx.doi.org/10.1371/currents.RRN1212

https://tree.opentreeoflife.org/curator/study/view/pg_1428?tab=trees&tree=tree2855
Meredith, R.W., Janecka J., Gatesy J., Ryder O.A., Fisher C., Teeling E., Goodbla A., Eizirik E., Simao T., Stadler T., Rabosky D., Honeycutt R., Flynn J., Ingram C., Steiner C., Williams T., Robinson T., Herrick A., Westerman M., Ayoub N., Springer M., & Murphy W. 2011. Impacts of the Cretaceous Terrestrial Revolution and KPg Extinction on Mammal Diversification. Science 334 (6055): 521-524.
http://dx.doi.org/10.1126/science.1211028

https://tree.opentreeoflife.org/curator/study/view/ot_168?tab=trees&tree=tree1
Almeida, Francisca C., Norberto P. Giannini, Nancy B. Simmons, Kristofer

In [9]:
#wget https://files.opentreeoflife.org/synthesis/opentree12.3/output/grafted_solution/grafted_solution.tre
fi = open("grafted_solution.tre").readlines()
for lin in fi:
    lin = lin.replace('(',',')
    lin = lin.replace(')',',')
    lii = lin.split(',')
    synth_ottids = set(lii)

In [10]:
pruned = copy.deepcopy(output.tree)

In [11]:
taxa_to_retain = []
for leaf in pruned.leaf_nodes():
    tax = leaf.taxon
    if tax:
        ottid = tax.label.split(' ')[-1]
   # elif leaf.label:
   #     ottid = leaf.label.split('_')[-1]
    else:
        ottid = None
    if ottid in synth_ottids:
        taxa_to_retain.append(tax)

In [12]:
pruned.retain_taxa(taxa_to_retain)

In [13]:
# Some characters in names mess up newick tree readers
def remove_problem_characters(instr, prob_char = "():#", replace_w = '?'):
    problem_characters = set(prob_char)
    for char in problem_characters:
        instr = instr.replace(char,replace_w)
    return instr

In [14]:
#Some of these taxon names are still causing issues...
for taxon in pruned.taxon_namespace:
    taxon.label = remove_problem_characters(taxon.label)

for node in pruned:
    if node.label:
        node.label = remove_problem_characters(node.label)

In [15]:
pruned.write(path="bats_pruned.tre",schema="newick")


In [27]:
fi.write("Name, ott_id, gbif_id, ncbi_id, synonyms")
for tip in taxa_to_retain:
    ott_id = tip.label.split()[-1].strip('ott')
    resp = OT.taxon_info(int(ott_id))

ERROR:root:Error in POST to https://api.opentreeoflife.org/v3/taxonomy/taxon_info
Traceback (most recent call last):
  File "/home/ejmctavish/snap/jupyter/common/lib/python3.7/site-packages/opentree/ws_wrapper.py", line 166, in _call_api
    call_obj = self._http_request(url, http_method, data=data, headers=headers)
  File "/home/ejmctavish/snap/jupyter/common/lib/python3.7/site-packages/opentree/ws_wrapper.py", line 231, in _http_request
    allow_redirects=True)
  File "/home/ejmctavish/snap/jupyter/common/lib/python3.7/site-packages/requests/api.py", line 60, in request
    return session.request(method=method, url=url, **kwargs)
  File "/home/ejmctavish/snap/jupyter/common/lib/python3.7/site-packages/requests/sessions.py", line 533, in request
    resp = self.send(prep, **send_kwargs)
  File "/home/ejmctavish/snap/jupyter/common/lib/python3.7/site-packages/requests/sessions.py", line 646, in send
    r = adapter.send(request, **kwargs)
  File "/home/ejmctavish/snap/jupyter/common/l

KeyboardInterrupt: 

In [28]:
resp.response_dict

{'flags': [],
 'is_suppressed': False,
 'is_suppressed_from_synth': False,
 'name': 'Murina peninsularis',
 'ott_id': 772161,
 'rank': 'species',
 'source': 'ott3.2draft9',
 'synonyms': ['Murina cyclotis peninsularis'],
 'tax_sources': ['ncbi:685783', 'gbif:8230112'],
 'unique_name': 'Murina peninsularis'}

In [31]:
fi=open("bat_tax_info.tsv","w")
fi.write("SppName\tLabel\tOttId\tGbifId\tncbiId\tSynonyms\n")
for tip in taxa_to_retain:
    ott_id = tip.label.split()[-1].strip('ott')
    resp = OT.taxon_info(int(ott_id))    
    match = resp.response_dict
    spp = match['name']
    gbif_id = ""
    ncbi_id = ""
    for source in match['tax_sources']:
        if source.startswith("gbif"):
            gbif_id = gbif_id + source
        if source.startswith("ncbi"):
            ncbi_id = ncbi_id + source
    synonyms = ",".join(match['synonyms'])
    fi.write("{}\t{}\t{}\t{}\t{}\t{}\n".format(spp, tip.label, ott_id, gbif_id, ncbi_id, synonyms))
fi.close()